In [ ]:
from logging import Logger
log = Logger("tasks.py")
log.log(0, "Hello")

In [2]:
import tasks

if __name__ == "__main__":

    db = "Orders_RELATIONAL_DB"
    schema = "dbo"
    raw_data = "raw_data_source.xlsx"

    conn_ER = tasks.connect()

    tasks.create_database(conn_ER, db)
    
    tasks.drop_constraints_if_exist(conn_ER, db, schema)
    
    for table in tasks.TABLES:    
        tasks.drop_table(conn_ER, table, db, schema)
        tasks.create_table(conn_ER, table, db, schema)
        tasks.insert_into_table(conn_ER, raw_data, table, db, schema)
    
    tasks.set_constraints(conn_ER, db, schema)

    conn_ER.close()


AttributeError: module 'tasks' has no attribute 'TABLES'

In [3]:
import tasks
from utils import parse_config

if __name__ == "__main__":

    config = parse_config("config")

    db = config.get("db")
    schema = config.get("schema")
    raw_data = config.get("raw_data")
    TABLES = config.get("TABLES")

    conn_ER = tasks.connect()

    tasks.create_database(conn_ER, db)

    tasks.drop_constraints_if_exist(conn_ER, db, schema)

    for table in TABLES:
        tasks.drop_table_if_exists(conn_ER, table, db, schema)
        tasks.create_table(conn_ER, table, db, schema)
        tasks.insert_into_table(conn_ER, raw_data, table, db, schema)

    tasks.set_constraints(conn_ER, db, schema)

    conn_ER.close()

    #     conn_Dim = tasks.connect_db_create_cursor("Database2")
    #     tasks.drop_table(conn_Dim, 'dim_people_scd1', 'Orders_DW', 'dbo')
    #     tasks.create_table(conn_Dim, 'dim_people_scd1', 'Orders_DW', 'dbo')
    #     tasks.update_dim_table(conn_Dim, 'dim_people_scd1', 'Orders_DW', 'dbo',
    #                            'people', 'Orders_ER', 'dbo')

    #     conn_ER.close()
    #     conn_Dim.close()




The dbo.Categories table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Categories table from the database Orders_RELATIONAL_DB has been created
Inserting into Categories...


8 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Categories table

The dbo.Customers table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Customers table from the database Orders_RELATIONAL_DB has been created
Inserting into Customers...


91 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Customers table

The dbo.Employees table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Employees table from the database Orders_RELATIONAL_DB has been created
Inserting into Employees...
9 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Employees table

The dbo.Suppliers table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Suppliers table from the database Orders_RELATIONAL_DB has been created
Inserting into Suppliers...


29 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Suppliers table

The dbo.Products table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Products table from the database Orders_RELATIONAL_DB has been created
Inserting into Products...


77 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Products table

The dbo.Region table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Region table from the database Orders_RELATIONAL_DB has been created
Inserting into Region...
4 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Region table

The dbo.Shippers table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Shippers table from the database Orders_RELATIONAL_DB has been created
Inserting into Shippers...
3 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Shippers table

The dbo.Territories table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Territories table from the database Orders_RELATIONAL_DB has been created
Inserting into Territories...


53 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Territories table

The dbo.Orders table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Orders table from the database Orders_RELATIONAL_DB has been created
Inserting into Orders...


830 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Orders table

The dbo.OrderDetails table from the database Orders_RELATIONAL_DB has been dropped
The dbo.OrderDetails table from the database Orders_RELATIONAL_DB has been created
Inserting into OrderDetails...


2155 rows have been inserted into the Orders_RELATIONAL_DB.dbo.OrderDetails table

Added primary key constraints
Added foriegn key constraints
